### 一、`MolecularGCN` 类解读

#### 关于 `torch.nn.Module`

`nn.module` 是 PyTorch 中所有神经网络模块的基类。它封装了参数管理、子模块管理等功能，使得构建和训练神经网络更加方便。

**核心功能**

- 参数管理：自动追踪和管理模型中所有可学习的参数（如权重和偏置）
- 子模块嵌套：允许一个模块包含多个子模块，支持层次化网络结构
- 前向传播定义：必须实现 `forward(self, *input)` 方法，定义数据如何通过模型
- 模式切换：提供 `.train()` 和 `.eval()` 方法，用于切换训练和评估模式（影响 Dropout、BatchNorm 等层的行为）
- 设备迁移：通过 `.to(device)` 方法可将模型整体移动到 GPU 或 CPU

#### `__init__()`

**参数**：`input_dim, hidden_dim, output_dim, dropout_rate, activation, use_bactch_norm, use_residual, attention_heads`

- 定义了模型的超参数，可在创建模型实例时自定义这些值

**初始化过程**

- `super(MolecularGCN, self).__init__()`: 标准操作，调用父类 nn.Module 的构造函数，确保 PyTorch 的底层机制（如参数追踪）能正常工作
- 根据传入的字符串（如 'relu'）将 `self.activation` 设置为对应的 PyTorch 函数（如 `F.relu`）。这样在 `forward` 方法中就可以直接调用 `self.activation(x)`，而无需重复写 `if-else` 判断

**核心网络层构建**

1. `GCN` & `GAT` (模型的核心特征)

```Python
self.gcn_layers = nn.ModuleList()
self.gat_layers = nn.ModuleList()

# Input layer
self.gcn_layers.append(GCNConv(input_dim, hidden_dims[0]))
self.gat_layers.append(GATConv(hidden_dims[0], hidden_dims[0] // attention_heads,
                               heads=attention_heads, dropout=dropout_rate))

# Hidden layers
for i in range(1, len(hidden_dims)):
    self.gcn_layers.append(GCNConv(hidden_dims[i-1], hidden_dims[i]))
    self.gat_layers.append(GATConv(hidden_dims[i], hidden_dims[i] // attention_heads,
                                  heads=attention_heads, dropout=dropout_rate))
```

- `nn.ModuleList`: 有点类似 Python 列表，能正确地注册其中包含的所有 `nn.Module`，以便 PyTorch 能追踪它们的参数
- 架构模式：对于每一层，它都先应用一个 `GCNConv`（图卷积层），然后紧接着应用一个 `GATConv`（图注意力层）
    - `GCNConv`：擅长聚合邻居信息，考虑**图的拓扑结构**
    - `GATConv`：为不同的邻居分配不同的权重（注意力），能**学习哪些邻居更重要**
- 数据流：`输入 input_dim` -> `GCN` -> `GAT` -> `输出 hidden_dims[0]` -> `GCN` -> `GAT` -> `输出 hidden_dims[1]` -> `...`
- GAT 维度计算：`GATConv` 的 `out_channels` 参数是 `hidden_dims[i] // attention_heads`。因为 `heads` 个头的输出会被拼接起来，所以最终输出维度是 `(hidden_dims[i] // attention_heads)` * `attention_heads` = `hidden_dims[i]`，与 GCN 层的输出维度保持一致

2. **批归一化层**

```python
if use_batch_norm:
    self.batch_norms = nn.ModuleList()
    for dim in hidden_dims:
        self.batch_norms.append(nn.BatchNorm1d(dim))
```

- 如果 `use_batch_norm` 为 `True`，就为每一个隐藏层创建一个 `BatchNorm1d` 层。它通常在激活函数之前应用，用于归一化每一层的输入，使训练更稳定、更快速

3. **注意力池化层**

```python
self.attention_pooling = AttentionPooling(hidden_dims[-1])
```

- GCN/GAT 层输出的是每个节点的特征（维度 `(num_nodes, hidden_dims[-1])`）。但我们的目标是预测整个分子的属性（一个值），所以需要一个图级的表示
- `AttentionPooling` 是一种可学习的池化方式。它会为每个节点学习一个重要性权重，然后将所有节点的特征加权求和，得到一个固定的图级向量（维度 1, `(hidden_dims[-1])`）

4. **预测头**: 标准的多层感知机（`MLP`），负责将图级特征转换为最终的预测值

```python
self.prediction_head = nn.Sequential(
    nn.Linear(hidden_dims[-1] * 3, hidden_dims[-1] // 2),  # 3 pooling types concatenated
    nn.Dropout(dropout_rate),
    self._get_activation_layer(),
    nn.Linear(hidden_dims[-1] // 2, output_dim)
)
```

- `hidden_dims[-1] * 3`: 关键细节。在进入预测头之前，三种不同的池化结果（可能是注意力池化、平均池化、最大池化）被拼接在了一起，通过拼接这三种不同统计特性的表示，模型可以从不同角度总结图的信息，从而做出更鲁棒的预测

**权重初始化**

```python
self._initialize_weights()
```

- 调用一个自定义的权重初始化方法。良好的权重初始化（如 `Xavier` 或 `He` 初始化）对模型的收敛至关重要，可以打破对称性，避免梯度消失或爆炸

#### `_get_activation_layer(self)`

```python
def _get_activation_layer(self):
    """Get activation layer based on configuration."""
    if isinstance(self.activation, type(F.relu)):
        return nn.ReLU()
    elif isinstance(self.activation, type(F.gelu)):
        return nn.GELU()
    elif isinstance(self.activation, type(F.leaky_relu)):
        return nn.LeakyReLU(0.2)
    else:
        return nn.ReLU()
```

- 前置知识：在 PyTorch 中，激活函数有两种形式：
    - **函数式**：如 `torch.nn.functional.relu`。它直接对输入张量进行操作，不包含可学习的参数。在 `__init__` 中，我们将 `self.activation` 设置为这种形式，以便在 `forward` 中灵活调用
    - **模块式**：如 `torch.nn.ReLU`。它是一个 `nn.Module`，可以被添加到 `nn.Sequential` 容器中

在 `__init__` 的 `prediction_head` 中，我们使用了 `nn.Sequential`，它要求其内部所有组件都是 `nn.Module`。因此，我们需要一个方法将函数式的激活函数“转换”成模块式的激活层，该函数的工作流程为：
- （以其中一个 `if-else` 为例）`isinstance(self.activation, type(F.relu))`：检查 self.activation 是否是 F.relu 这个函数的类型
- 根据判断结果，返回一个相应的模块实例，如 `nn.ReLU()`、`nn.GELU()` 等
- else 分支提供了一个默认的 nn.ReLU()，增加了代码的健壮性

#### `_initialize_weights(self)` - 权重初始化函数

```python
def _initialize_weights(self):
    """Initialize model weights."""
    for m in self.modules():
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.ones_(m.weight)
            nn.init.zeros_(m.bias)
```

**核心作用**: 为模型中的所有参数（权重和偏置）设定一个合理的初始值
**工作流程**
- `for m in self.modules()`: `self.modules()` 会递归地遍历模型中的所有子模块（包括 `prediction_head` 里的 `Linear` `层、gcn_layers` 等）
- 对于 `nn.Linear` 层：
    - `nn.init.xavier_uniform_(m.weight)`：使用 `Xavier` 均匀初始化，旨在让每一层输入和输出的方差保持一致，从而防止信号在深层网络中过快地消失或爆炸
    - `nn.init.zeros_(m.bias)`：将偏置项初始化为零。这是一种常见且安全的做法
- 对于 `nn.BatchNorm1d` 层：
    - `nn.init.ones_(m.weight)`：将 BN 层的缩放因子（`gamma`）初始化为1
    - `nn.init.zeros_(m.bias)`：将 BN 层的偏移因子（`beta`）初始化为0
    - 这意味着在训练开始时，BN 层不改变其输入（恒等变换），有助于稳定训练初期的过程

#### `forward(self, data: Data) -> torch.Tensor` - 前向传播函数（**模型核心**）

```python
def forward(self, data: Data) -> torch.Tensor:
    # 1. 输入解包
    x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
    x_in = x # 保存初始输入，用于第一个残差连接

    # 2. 循环通过 GNN 层
    for i, (gcn_layer, gat_layer) in enumerate(zip(self.gcn_layers, self.gat_layers)):
        # GCN 层
        x = gcn_layer(x, edge_index, edge_attr)

        # 批归一化
        if self.use_batch_norm and i < len(self.batch_norms):
            x = self.batch_norms[i](x)

        # 激活函数和 Dropout
        x = self.activation(x)
        x = F.dropout(x, p=self.dropout_rate, training=self.training)

        # GAT 层
        x_att = gat_layer(x, edge_index)
        x = x + x_att  # 将 GAT 的输出作为残差加到 GCN 的输出上

        # 跨层残差连接
        if self.use_residual and i < len(self.gcn_layers) - 1:
            if x.size(-1) == x_in.size(-1): # 只有维度相同时才能相加
                x = x + x_in
            x_in = x # 更新 x_in 为下一层做准备

    # 3. 全局池化
    graph_embedding = self.attention_pooling(x, batch)  # 可学习的注意力池化
    mean_pool = global_mean_pool(x, batch)              # 平均池化
    max_pool = global_max_pool(x, batch)                # 最大池化
    add_pool = global_add_pool(x, batch)                # 求和池化

    # 4. 组合池化结果
    combined_pool = torch.cat([graph_embedding, mean_pool, max_pool, add_pool], dim=1)

    # 5. 最终预测
    output = self.prediction_head(combined_pool)

    return output.squeeze(-1) # 移除最后一个维度，使其形状为 [batch_size]
```

1. **输入解包**：

- 从 PyTorch Geometric 的 `Data` 对象中提取节点特征 `x`、边索引 `edge_index`、边属性 `edge_attr` 和批次向量 `batch`
- `x_in = x`：为残差连接保存原始输入

2. **GNN 层循环（核心计算步骤）**

- 数据依次通过每一对 `(GCN, GAT)` 层
- `GCN -> BN -> Activation -> Dropout`：这是一个非常标准的 GNN 层流程
- `GAT` 层：在 `GCN` 处理后，再通过一个 `GAT` 层来引入注意力机制
- `x = x + x_att`: 将 `GAT` 层的输出视为对 `GCN` 输出的“修正”或“增强”，通过**残差连接**的方式融合起来
- 跨层残差连接：`if self.use_residual` 块实现了标准的 `ResNet` 风格的残差连接。它**将当前层的输出 `x` 与该层循环开始前的输入 `x_in` 相加**。`if x.size(-1) == x_in.size(-1)` 是一个关键的安全检查，确保只有在维度匹配时才进行相加

3. **全局池化**

- 经过所有 `GNN` 层后得到了每个节点的最终特征表示 `x`
- 为了得到整个图的表示，代码应用了四种不同的池化策略：
    - `AttentionPooling`: 可学习的池化
    - `global_mean_pool`: 所有节点特征的平均值
    - `global_max_pool`: 所有节点特征的最大值
    - `global_add_pool`: 所有节点特征的和

4. **组合池化结果**

- `torch.cat(...)`: 将四种池化得到的图级向量沿着特征维度（`dim=1`）拼接起来，形成一个信息非常丰富的、更大的图表示向量

5. **最终预测**

- `output = self.prediction_head(combined_pool)`: 将组合后的向量送入预测头（一个 `MLP`），得到最终的预测值
- `return output.squeeze(-1)`: `prediction_head` 的输出形状是 `[batch_size, 1]`。对于回归任务，我们通常希望输出形状是 `[batch_size]`。`squeeze(-1)` 移除了这个多余的维度，使得输出可以直接用于计算损失（如 `MSELoss`）

### 二、`AttentionPooling` 类解读

#### 1. 核心思想

传统的池化方法，如平均池化（取所有节点特征的平均值）或最大池化（取所有节点特征的最大值），平等地对待所有节点。这在很多场景下是不合理的。

以分子为例，一个分子的生物活性可能主要由其核心官能团（如某个特定的原子或化学键）决定，而其他部分（如长长的碳链）可能影响较小。

注意力池化就是为了解决这个问题。它**通过一个小型神经网络（`self.attention`）为每个节点学习一个重要性权重**。最终的图表示是所有节点特征的加权平均值，权重就是学习到的重要性分数。

#### 2. `__init__`

```python
def __init__(self, hidden_dim: int):
    super(AttentionPooling, self).__init__()

    self.attention = nn.Sequential(
        nn.Linear(hidden_dim, hidden_dim // 2),
        nn.Tanh(),
        nn.Linear(hidden_dim // 2, 1),
        nn.Softmax(dim=0)
    )
```

- `nn.Sequential`: 一个容器，将多个层按顺序串联起来
- `nn.Linear(hidden_dim, hidden_dim // 2)`: 第一个线性层。它将每个节点的 `hidden_dim` 维特征映射到一个更低的维度（`hidden_dim // 2`）。这是一种常见的降维技巧，可以减少计算量并增加非线性
- `nn.Tanh()`: 将输出值压缩到 [-1, 1] 之间，引入非线性，有助于模型学习更复杂的模式
- `nn.Linear(hidden_dim // 2, 1)`: 第二个线性层。它将处理后的特征映射到一个标量（一个数值）。这个值就是该节点的原始重要性分数
- `nn.Softmax(dim=0)`: Softmax 函数能将一组数值转换成概率分布（所有值相加为1）。`dim=0` 表示沿着第一个维度（节点维度）进行归一化。注意：在 `__init__` 中直接使用 `Softmax(dim=0)` 是基于**输入 `self.attention` 的张量只包含单个图的所有节点**的假设。但**在实际批处理中，输入会包含多个图的节点**。因此，在 `forward` 方法中，我们需要一个更精细的归一化过程来覆盖或修正这一冲突

#### 3. `forward`

**参数**

- `x`: 批次中所有节点的特征矩阵。（形状 `[num_nodes, hidden_dim]`）
- `batch`: 分配每个节点对应哪个图（如 `batch = [0, 0, 0, 1, 1, 1, 1, 1, 1, 1]` 表示前3个节点属于图0，后7个节点属于图1）

**逐步计算流程**

1. 计算原始注意力分数

```python
attention_weights = self.attention(x)       # [num_nodes, 1]
```

- 将所有节点特征输入到评分器网络，得到每个节点的原始分数。形状是 `[num_nodes, 1]`
- **注意**：这里的 `Softmax(dim=0)` 是在所有 `num_nodes` 上全局归一化的，这并不是我们想要的。我们希望在每个图内部进行归一化。接下来的步骤就是为了修正这一点

2. 创建批次掩码

```python
    batch_size = batch.max().item() + 1
    batch_mask = F.one_hot(batch, num_classes=batch_size).float()  # [num_nodes, batch_size]
```

- `batch_size`：计算出这个批次里有多少个图
- `F.one_hot(...)`：创建一个独热编码的掩码矩阵。如果 `batch = [0, 0, 1]`，`batch_mask` 就是 ```[[1, 0], [1, 0], [0, 1]]```。`num_classes=batch_size` 表示每个节点对应哪个图，因此 `num_classes` 等于 `batch_size`。`float()` 将掩码矩阵转换成浮点数

3. 应用掩码并重新归一化

```python
    attention_weights = attention_weights * batch_mask
    attention_sums = attention_weights.sum(dim=0, keepdim=True)
    attention_weights = attention_weights / (attention_sums + 1e-8)
```

- `attention_weights * batch_mask`: 将全局的注意力分数与掩码相乘。这样，不属于某个图的节点的分数在该图对应的列上就变成了0。
- `attention_weights.sum(dim=0, keepdim=True)`: 按列求和，得到每个图内部所有节点的分数总和。形状变为 `[1, batch_size]`。
- `attention_weights / (attention_sums + 1e-8)`: 用每个节点的分数除以其所在图的总分。这一步**真正实现了在每个图内部的归一化**，使得每个图的所有节点的注意力权重之和为1。`+ 1e-8` 是为了防止除以零。

现在，`attention_weights` 的形状是 `[num_nodes, batch_size]`，其中 `attention_weights[i, j]` 表示节点 `i` 对图 `j` 的归一化后的注意力权重。

4. 加权求和得到图嵌入

```python
graph_embedding = torch.matmul(attention_weights.t(), x)  # [batch_size, hidden_dim]
```

通过矩阵乘法完成加权求和。
- `attention_weights.t()`: 转置注意力权重矩阵，形状变为 `[batch_size, num_nodes]`。
- `x`: 节点特征矩阵，形状 `[num_nodes, hidden_dim]`。
- `torch.matmul(...)`: 矩阵乘法 `[batch_size, num_nodes] @ [num_nodes, hidden_dim]` 的结果是 `[batch_size, hidden_dim]`。

这个结果的每一行 `j` 就是对图 `j` 的向量表示，它是通过将图 `j` 中所有节点的特征 `x` 与其对应的注意力权重相乘后求和得到的。

### 三、`MultiTaskMolecularGCN` 类解读

#### 核心思想：多任务学习，同时预测多指标

1. **共享主干**：模型的前半部分（在这里是所有的 GNN 层和池化层）是所有任务共享的。它的目标是学习一个通用的、高质量的分子表示（一个向量），这个向量捕捉了与所有任务相关的分子特征
2. **任务特定头**：模型的后半部分为每个任务设置一个独立的、小的神经网络（称为“头”）。每个头接收共享的分子表示，并专门负责输出对应任务的预测结果

#### `__init__`

```python
def __init__(
    self,
    # ... 其他参数 ...
    output_dims: Dict[str, int] = {"pic50": 1, "logp": 1, "solubility": 1},
    **kwargs
):
```

**关键变化**

- `output_dims: Dict[str, int]`: 这是与单任务模型最根本的区别。它不再是一个单一的整数 `output_dim`，而是一个字典。键是任务名称（如 `pic50`），值是该任务的输出维度（如 1 表示回归一个值）。这使得模型非常灵活，可以轻松地增减任务或处理不同类型的输出（例如，一个分类任务的输出维度可能是类别数）

**初始化流程**

1. 调用父类构造函数

```python
super(MultiTaskMolecularGCN, self).__init__(
    input_dim=input_dim,
    hidden_dims=hidden_dims,
    output_dim=hidden_dims[-1] // 2,
    **kwargs
)
```

- 它调用了 `MolecularGCN` 的 `__init__`，复用了所有 GNN 层、池化层等的创建逻辑。
- **关键点**：将父类的 `output_dim` 设置为 `hidden_dims[-1] // 2`，原因：在多任务模型中，父类的 `prediction_head` 不再用于最终预测，而是被“借用”来生成一个**共享嵌入向量**。这个向量的维度就是 `hidden_dims[-1] // 2`。

2. 存储任务信息

```python
self.tasks = list(output_dims.keys())
self.output_dims = output_dims
```

- 将任务名称列表和输出维度字典保存为实例属性，方便在 `forward` 中使用。

3. 创建任务特定头

```python
self.task_heads = nn.ModuleDict()
for task, dim in output_dims.items():
    self.task_heads[task] = nn.Sequential(...)
```

- `nn.ModuleDict`：类似 Python 字典，但能正确地注册其中包含的所有 `nn.Module`。这样，优化器才能找到并更新这些任务头中的参数。
- **循环创建**：代码遍历 `output_dims` 字典，为**每一个任务**都创建一个独立的 `nn.Sequential` 网络（即“头”）。
- **头的结构**：每个头都是一个简单的 MLP，它接收共享嵌入（维度 `hidden_dims[-1] // 2`），并输出该任务所需的维度 `dim`。

4. `forward`

a. 复用父类逻辑

```python
# Get shared graph embedding (reusing parent class logic)
x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch
# ... (与父类 forward 完全相同的 GNN 层循环) ...
# Global pooling
# ... (与父类 forward 完全相同的池化代码) ...
combined_pool = torch.cat([graph_embedding, mean_pool, max_pool, add_pool], dim=1)
```

b. 生成共享嵌入

```python
shared_embedding = self.prediction_head[0](combined_pool)
```

- `self.prediction_head` 是在父类中定义的 `nn.Sequential`。这里只取它的**第一层**（`self.prediction_head[0]`），它是一个 `nn.Linear` 层。
- 将拼接后的池化向量（维度 `hidden_dims[-1] * 4`）映射到之前设定的共享嵌入维度（`hidden_dims[-1] // 2`）。
- `shared_embedding` 即为模型学习到的、对所有任务都有用的通用分子表示。

c. 分派到各个任务头

```python
# Task-specific predictions
predictions = {}
for task in self.tasks:
    task_output = self.task_heads[task](shared_embedding)
    predictions[task] = task_output.squeeze(-1)
return predictions
```

- 创建一个空字典 `predictions` 用于存储结果。
- **循环遍历所有任务**：对于每一个任务名 `task`：
    - 从 `self.task_heads` 中取出对应的任务头。
    - 将 `shared_embedding` 输入到这个任务头中，得到该任务的原始输出。
    - `squeeze(-1)`：和单任务模型一样，将输出形状从 `[batch_size, 1]` 调整为 `[batch_size]`。
    - 将结果存入 `predictions` 字典，键为任务名。
- 最后返回这个包含所有任务预测结果的字典。